# DATA SCRAPPING

In [3]:
import pandas as pd

df = pd.read_csv('anggota_dpr_2024-2029.csv')

print(df.columns)

Index(['No. Anggota', 'Daerah pemilihan', 'Potret', 'Nama', 'Komisi', 'Fraksi',
       'Fraksi.1', 'Perolehan Suara Sah', 'Catatan'],
      dtype='object')


# HAPUS KOLOM Potret, Fraksi, & Perolehan Suara Sah

In [4]:
df = df.drop(columns=['Potret', 'Fraksi', 'Perolehan Suara Sah'])

print(df.head())

   No. Anggota Daerah pemilihan                               Nama  \
0            1           Aceh I            H. Irmawan S.Sos., M.M.   
1          155           Aceh I        Jamaluddin Idham S.H., M.H.   
2          264           Aceh I  Drs. H. T. Zulkarnaini Ampon Bang   
3          366           Aceh I          H. Muslim Ayub S.H., M.M.   
4          435           Aceh I                            Ghufran   

        Komisi Fraksi.1 Catatan  
0     Komisi V      PKB     NaN  
1  Komisi XIII    PDI-P     NaN  
2   Komisi VII   Golkar     NaN  
3  Komisi XIII   NasDem     NaN  
4    Komisi VI      PKS     NaN  


# GANTI NAMA KOLOM Fraksi.1 MENJADI Fraksi

In [5]:
df = df.rename(columns={'Fraksi.1': 'Fraksi'})

print(df.head())

   No. Anggota Daerah pemilihan                               Nama  \
0            1           Aceh I            H. Irmawan S.Sos., M.M.   
1          155           Aceh I        Jamaluddin Idham S.H., M.H.   
2          264           Aceh I  Drs. H. T. Zulkarnaini Ampon Bang   
3          366           Aceh I          H. Muslim Ayub S.H., M.M.   
4          435           Aceh I                            Ghufran   

        Komisi  Fraksi Catatan  
0     Komisi V     PKB     NaN  
1  Komisi XIII   PDI-P     NaN  
2   Komisi VII  Golkar     NaN  
3  Komisi XIII  NasDem     NaN  
4    Komisi VI     PKS     NaN  


# Hapus nama anggota yang kolom Komisi-nya kosong. Karena itu nama-nama anggota DPR RI yang: mengundurkan diri, diangkat jadi menteri, meninggal dunia, dsb. Jadi nama anggota yang kolom Komisi-nya kosong itu sudah bukan anggota DPR RI lagi.

In [6]:
# Hapus baris di mana kolom 'Komisi' kosong (NaN atau string kosong)
df = df[df['Komisi'].notna()]  # menghapus NaN
df = df[df['Komisi'].str.strip() != '']  # menghapus string kosong atau spasi

# Cek hasilnya
print(df.head())
print(f"Jumlah baris setelah pembersihan: {len(df)}")

   No. Anggota Daerah pemilihan                               Nama  \
0            1           Aceh I            H. Irmawan S.Sos., M.M.   
1          155           Aceh I        Jamaluddin Idham S.H., M.H.   
2          264           Aceh I  Drs. H. T. Zulkarnaini Ampon Bang   
3          366           Aceh I          H. Muslim Ayub S.H., M.M.   
4          435           Aceh I                            Ghufran   

        Komisi  Fraksi Catatan  
0     Komisi V     PKB     NaN  
1  Komisi XIII   PDI-P     NaN  
2   Komisi VII  Golkar     NaN  
3  Komisi XIII  NasDem     NaN  
4    Komisi VI     PKS     NaN  
Jumlah baris setelah pembersihan: 579


# Buat kolom jabatan. Ganti nama kolom Catatan menjadi Jabatan. 

# Kemudian, hapus semua value yang cuma ada keterangannya saja.  Contoh, hapus [Ket. 1] atau [Ket. 2] atau [Ket. 12]. Semua yang ada ket itu hapus saja.

# Kemudian, semua value yang kosong di kolom Jabatan, diganti dengan tanda "-"

In [7]:
df.rename(columns={'Catatan': 'Jabatan'}, inplace=True)

import re

df['Jabatan'] = df['Jabatan'].apply(lambda x: None if pd.notna(x) and re.fullmatch(r'\[Ket\. ?\d+\]', x.strip()) else x)

df['Jabatan'] = df['Jabatan'].fillna('-')
df['Jabatan'] = df['Jabatan'].apply(lambda x: '-' if str(x).strip() == '' else x)

print(df[['Nama', 'Jabatan']].head(10))

                                 Nama  \
0             H. Irmawan S.Sos., M.M.   
1         Jamaluddin Idham S.H., M.H.   
2   Drs. H. T. Zulkarnaini Ampon Bang   
3           H. Muslim Ayub S.H., M.M.   
4                             Ghufran   
5               H. Nazaruddin Dek Gam   
7            H. T. Ibrahim S.T., M.M.   
8                      H. Ruslan Daud   
9            Ir. H. T. A. Khalid M.M.   
10                  H. Ilham Pangestu   

                                         Jabatan  
0                                              -  
1                                              -  
2                                              -  
3                                              -  
4                                              -  
5         Ketua Mahkamah Kehormatan Dewan DPR RI  
7                                    [Ket. 2][2]  
8                                              -  
9                                              -  
10  Wakil Ketua Badan Urusan Rumah Tan

# Membuat tabel "Provinsi" agar dapat membuat chart map di Looker Studio

In [29]:
import re

romawi = {'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI'}

def ambil_provinsi(dapil):
    parts = dapil.strip().split()
    if parts[-1].upper() in romawi:
        return ' '.join(parts[:-1])
    else:
        return dapil.strip()

df['Provinsi'] = df['Daerah pemilihan'].apply(ambil_provinsi)

df.head()

,No. Anggota,Daerah pemilihan,Nama,Komisi,Fraksi,Jabatan,Provinsi,Koalisi
0,1,Aceh I,"H. Irmawan S.Sos., M.M.",Komisi V,PKB,-,Aceh,Oposisi
1,155,Aceh I,"Jamaluddin Idham S.H., M.H.",Komisi XIII,PDI-P,-,Aceh,Oposisi
2,264,Aceh I,Drs. H. T. Zulkarnaini Ampon Bang,Komisi VII,Golkar,-,Aceh,Koalisi Pemerintah
3,366,Aceh I,"H. Muslim Ayub S.H., M.M.",Komisi XIII,NasDem,-,Aceh,Oposisi
4,435,Aceh I,Ghufran,Komisi VI,PKS,-,Aceh,Oposisi


# Memasukkan Kolom Oposisi ke dalam tabel

# Kolom Koalisi adalah hasil kategorisasi fraksi anggota DPR RI berdasarkan posisi partai politik mereka dalam pemerintahan Presiden Prabowo Subianto dan Wakil Presiden Gibran Rakabuming Raka untuk periode 2024–2029.

# Kategori ini hanya terdiri dari dua nilai:

# -Koalisi Pemerintah: Fraksi partai politik yang tergabung dalam kabinet pemerintahan Prabowo-Gibran.

# -Oposisi: Fraksi partai politik yang tidak bergabung dalam pemerintahan dan cenderung bersikap kritis terhadap kebijakan pemerintah.

# *Catatan Konteks: Klasifikasi ini didasarkan pada konstelasi politik awal pemerintahan Prabowo-Gibran (hingga Mei 2025). Karena dinamika politik di Indonesia sangat cair, komposisi koalisi dapat berubah sepanjang periode pemerintahan. 

In [30]:
koalisi_pemerintah = [
    'Gerindra', 'Golkar', 'PAN', 'Demokrat',
    'PSI', 'PBB', 'Gelora', 'Garuda', 'Prima'
]

def tentukan_koalisi(fraksi):
    if fraksi in koalisi_pemerintah:
        return 'Koalisi Pemerintah'
    else:
        return 'Oposisi'

df['Koalisi'] = df['Fraksi'].apply(tentukan_koalisi)

df.head()

,No. Anggota,Daerah pemilihan,Nama,Komisi,Fraksi,Jabatan,Provinsi,Koalisi
0,1,Aceh I,"H. Irmawan S.Sos., M.M.",Komisi V,PKB,-,Aceh,Oposisi
1,155,Aceh I,"Jamaluddin Idham S.H., M.H.",Komisi XIII,PDI-P,-,Aceh,Oposisi
2,264,Aceh I,Drs. H. T. Zulkarnaini Ampon Bang,Komisi VII,Golkar,-,Aceh,Koalisi Pemerintah
3,366,Aceh I,"H. Muslim Ayub S.H., M.M.",Komisi XIII,NasDem,-,Aceh,Oposisi
4,435,Aceh I,Ghufran,Komisi VI,PKS,-,Aceh,Oposisi


# Save Data

In [31]:
df.to_csv('data_scraping_1.csv', index=False)

# Tahapan selanjutnya adalah melakukan cleaning data dari file tersebut secara manual menggunakan excel 

# Kemudian, akan ditambahkan kolom "Jenis Kelamin" yang juga akan dilakukan secara manual di excel